In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from math import cos, sin, exp, sqrt, pi, e
from datetime import datetime
from pandas import DataFrame

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset

mypalette = sns.color_palette(['#162f4d']) # https://www.hsluv.org/
sns.set_palette(mypalette)

## Constants to be used across the notebook

In [3]:
# Min and Max temperatures to be received by the user.
# Values [40,...,60]
Tmin_i = float(input("Enter min setpoint temperature: "))
Tmax_i = float(input("Enter max setpoint temperature: "))

# Domestic hot water temperature.
# Values [45,...,55]
Tdhw = float(input("Enter domestic hot water temperature: "))

# Our model supposes that the system has two different sources of heat.
# Values [20,...,30]
Ts1 = float(input("Temperature of source 1: "))
Ts2 = float(input("Temperature of source 2: "))

# difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
# Values [1,...,10]
DT_evap = float(input("Temperature difference among the supply and return pipes: "))

Enter min setpoint temperature:  40
Enter max setpoint temperature:  60
Enter domestic hot water temperature:  55
Temperature of source 1:  23
Temperature of source 2:  29
Temperature difference among the supply and return pipes:  4


## Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [127]:
# Time repr
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")

s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

* s1_schedule_path='./data/private/s1_source_fake_schedule.xlsx'
* s1_schedule=array([[0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0]])
* s1_schedule[6,1]=0
* s1_schedule[7,1]=1



In [128]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

* s2_schedule_path='./data/private/s2_source_fake_schedule.xlsx'
* s2_schedule=array([[1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1, 1]])
* s2_schedule[15,1]=1
* s2_schedule[16,1]=0



## Read the dataset

In [4]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork'])

In [229]:
dataset_to_work = "rochester_newyork"
dataset_to_work

'rochester_newyork'

In [230]:
Tamb_hourly = AVAILABLE_DATASETS[dataset_to_work]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp
timestamp,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560
...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559


In [231]:
# Tamb_input_path = os.path.join(".", "data", "Ospitaletto", "processed", "Tamb_input.xls")
# Tamb_hourly_path = os.path.join(".", "data", "Ospitaletto", "processed", "Tamb_hourly.xls")

In [232]:
# temp=pd.read_excel(Tamb_input_path ,squeeze=True)
# Treal=pd.read_excel(Tamb_hourly_path, squeeze=True)

#Tamb_fit=pd.Series(temp)
#Tamb=pd.Series(Treal)

#Tamb.plot()
#Tamb_fit.plot(color='red')
#print('Tamb_min:',Tamb_fit.min(),'Tamb_max',Tamb_fit.max())


In [233]:
#date_rng = pd.date_range(start='1/1/2019 00:00:00', end='31/12/2019 00:00:00', freq='H')
#df = pd.DataFrame(date_rng, columns=['date'])

#df['Tamb_fit'] = Tamb_fit
#df['Tamb'] = Tamb


#df.head(12)
#df[df.index.month==12]



# Data manipulation-processing

In [234]:
#Tamb_hourly=pd.DataFrame(df, columns = ['date','Tamb','Tamb_fit']) #Ambient temperature on an hourly resolutionT


In [235]:
#Tamb_hourly['date'] =pd.to_datetime(Tamb_hourly['date'], infer_datetime_format=True) #Estoy asociando el index de tiempo a la columna Data-Ora
#Tamb_hourly=Tamb_hourly.set_index('date')

In [236]:
#Tamb_hourly.head()

In [237]:
# Tamb_hourly.info()

In [238]:
# Tamb_hourly.index

In [239]:
# Tamb_hourly.reset_index(inplace=True)

In [240]:
#Tamb_hourly['Month'] = Tamb_hourly.index.month


In [241]:
#Tamb_hourly

## Calculate Space Heating and Hot Water Temperature

The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [242]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [243]:
Tamb_hourly['space_heating_temp'] = Tamb_hourly.air_temp.apply(climatic_curve)
Tamb_hourly["hot_water_temp"] = Tdhw
#Tamb_hourly['Tsh'] = Tamb['Temp'].apply(lambda x: climatic_curve(x))
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp
timestamp,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,60.0,55.0
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,60.0,55.0
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,60.0,55.0
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,60.0,55.0
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,60.0,55.0
...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,60.0,55.0
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,60.0,55.0
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,60.0,55.0


In [244]:
# november_data = Tamb_hourly[Tamb_hourly.month == 11 ]
# november_data[["space_heating_temp","hot_water_temp"]].plot(figsize=(10,6), color=["#4e97a2",'#82272e'])

# plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

## Calculate User Temperature

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [245]:
def Tuser(fila):
    Tsh = fila.space_heating_temp
    Tdhw = fila.hot_water_temp
    month = fila.month
    
    #Las proporciones dependen de la localización
    if month==1:
        resultado= 0.98*Tsh+0.02*Tdhw
       
    elif month==2:
        resultado=0.96*Tsh+0.04*Tdhw
    
    elif month==3:
        resultado=0.94*Tsh+0.06*Tdhw
    
    elif month==4:
        resultado=0.90*Tsh+0.1*Tdhw
    
    elif month==5:
        resultado=0.75*Tsh+0.25*Tdhw
    
    elif month==6:
        resultado=0.25*Tsh+0.75*Tdhw
    
    elif month==7:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==8:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==9:
        resultado=0.1*Tsh+0.9*Tdhw
    
    elif month==10:
        resultado=0.80*Tsh+0.2*Tdhw
    
    elif month==11:
        resultado=0.95*Tsh+0.05*Tdhw
    
    elif month==12:
        resultado=0.97*Tsh+0.03*Tdhw
        
    return resultado

In [246]:
Tamb_hourly["user_temp"] = Tamb_hourly.apply(Tuser, axis=1)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp
timestamp,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,60.0,55.0,59.90
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,60.0,55.0,59.90
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,60.0,55.0,59.90
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,60.0,55.0,59.90
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,60.0,55.0,59.90
...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,60.0,55.0,59.85
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,60.0,55.0,59.85
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,60.0,55.0,59.85


In [247]:
# Tamb_hourly["user_temp"].plot(figsize=(10,6))

In [248]:
# Tamb_hourly["user_temp"].plot(figsize=(10,6))

In [249]:
# november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
# november_data["user_temp"].plot(figsize=(10,6))

# Network temperature 

In [250]:
# zz=1 # Depth [m]
# alpha=0.06048 # Ground thermal diffusivity, Banks [m^2/day]
# alpha_sec=7e-7 # Ground thermal diffusivity, Banks [m^2/s]
# t_sec=365*24*3600

# d_shift_max = Tamb_hourly.air_temp_fit.idxmax(axis=0) # Tamb_hourly['Tamb_fit'].idxmax(axis=0)
#dd_max= Tamb_hourly[Tamb_hourly.index == d_shift_max].dayofyear.item() # (Tamb_hourly.index.get_loc(d_shift_max))/24

# TODO: Guardar T_ave_fit y DT_y_fit en el dataframe. Tomar en cuenta que Tg_und y DT_y son constantes y no series.
#Tg_und = 9.19 #T_ave_fit #Undisturbed ground temperature
#DT_y= 12.78 #abs(DT_y_fit)



In [251]:
#def ground_temperature_hour(t,zz):
    #t is time in hours, but the calculation is done is seconds
#    T_banks= Tg_und+DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec)))*cos(2*pi/t_sec*(t-dd_max*24)*3600-zz*sqrt(pi/(alpha_sec*t_sec)))
    
#    return pd.Series(T_banks)

In [252]:
#Tamb_hourly = Tamb_hourly.reset_index(drop=False)
# Tamb_hourly

In [253]:
#Tamb_hourly.index

In [254]:
#Tamb_hourly["ground_temp_2"] = Tamb_hourly.apply(lambda fila: ground_temperature_hour(fila.hourofyear, zz), axis=1)
#Tamb_hourly

In [255]:
#Tamb_hourly[["Tground"]].plot()

In [256]:
#depth_aquifer= float(input("""Aquifer temperature depth:
#"""))

In [257]:
# fila.name because that's the index (numeric range from 0 to 8737)
# fila.name + 1 because the range starts in 0 and the function assumes it begins with 1.
#Tamb_hourly["Taq"] = Tamb_hourly.apply(lambda fila: ground_temperature_hour(fila.name + 1, depth_aquifer), axis=1)
#Tamb_hourly

In [258]:
#Tamb_hourly[["Taq"]].plot()

In [259]:
#Tamb_hourly[["Taq", "Tground"]].plot()

In [260]:
#Tamb_hourly = Tamb_hourly.set_index("date")
#Tamb_hourly

In [261]:
# Tamb_hourly

## Calculate Sources Temperature based on Working Hours

In [262]:
#def get_temp_by_working_hours(temp_src, datetime, schedule):
#    day_of_week = datetime.dayofweek
#    hour_of_day = datetime.hour
#    return temp_src * (schedule[hour_of_day, day_of_week])

In [263]:
dayofweeks = Tamb_hourly.index.dayofweek
hours = Tamb_hourly.index.hour

In [264]:
# Ts1 = Tamb_hourly.apply(lambda fila: get_temp_by_working_hours(Ts1, fila.name, s1_schedule), axis=1)
# Ts2 = Tamb_hourly.apply(lambda fila: get_temp_by_working_hours(Ts2, fila.name, s2_schedule), axis=1)
Tamb_hourly["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
Tamb_hourly["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp
timestamp,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,60.0,55.0,59.90,23.0,29.0
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,60.0,55.0,59.90,23.0,29.0
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,60.0,55.0,59.90,23.0,29.0
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,60.0,55.0,59.90,23.0,29.0
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,60.0,55.0,59.90,23.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,60.0,55.0,59.85,23.0,29.0
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,60.0,55.0,59.85,23.0,29.0
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,60.0,55.0,59.85,23.0,29.0


In [265]:
#Ts2 = Tamb_hourly.apply(lambda fila: get_temp_by_working_hours(Ts2, fila.name, s2_schedule), axis=1)
#Ts2.head(50)

In [266]:
#Tamb_hourly["source1_temp"] = Ts1
#Tamb_hourly["source2_temp"] = Ts2

#Tamb_hourly

In [267]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.mean([temp_s1, temp_s2])

In [268]:
Tamb_hourly["net_temp"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp
timestamp,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,60.0,55.0,59.90,23.0,29.0,26.0
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,60.0,55.0,59.90,23.0,29.0,26.0
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,60.0,55.0,59.90,23.0,29.0,26.0
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,60.0,55.0,59.90,23.0,29.0,26.0
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,60.0,55.0,59.90,23.0,29.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,60.0,55.0,59.85,23.0,29.0,26.0
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,60.0,55.0,59.85,23.0,29.0,26.0
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,60.0,55.0,59.85,23.0,29.0,26.0


In [269]:
# Tamb_hourly[["net_temp"]][Tamb_hourly.index < "2010-01-31"].plot()

# Coefficient of performance (COP)

In [270]:
DT_hx = 2.5
n_HP = 0.53
Te_o = Tamb_hourly.net_temp - DT_evap - DT_hx
Tc_o = Tamb_hourly.user_temp

Tamb_hourly['COP']= 1-n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP
timestamp,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,60.0,55.0,59.90,23.0,29.0,26.0,4.584604
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,60.0,55.0,59.85,23.0,29.0,26.0,4.588786
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,60.0,55.0,59.85,23.0,29.0,26.0,4.588786
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,60.0,55.0,59.85,23.0,29.0,26.0,4.588786


In [271]:
#Tamb_hourly[['COP']].plot(figsize=(10,6))
#Tc_o.plot()

In [272]:
#Tamb_hourly[['COP']].plot(figsize=(10,6))
#Tc_o.plot()

# Heat losses

In [273]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 2
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U / 1000 # Heat losses supply pipe [MWh]
    HL_r = (T_ret - T_gr) * U / 1000 # Heat losses return pipe [MWh]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [274]:
x = Tamb_hourly.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')

In [275]:
Tamb_hourly['E_loss_s'] = x['E_loss_s']
Tamb_hourly['E_loss_r'] = x['E_loss_r']
Tamb_hourly['E_loss_tot'] = x['E_loss_tot']
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,
2010-01-01 01:00:00,-2.9,1,1,2,-2.749240,2.752620,9.187559,60.0,55.0,59.90,23.0,29.0,26.0,4.584604,0.046495,0.038495,0.084990
2010-01-01 02:00:00,-3.1,1,1,3,-2.752504,2.748357,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604,0.046503,0.038503,0.085007
2010-01-01 03:00:00,-3.2,1,1,4,-2.755763,2.744097,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604,0.046512,0.038512,0.085024
2010-01-01 04:00:00,-3.3,1,1,5,-2.759015,2.739840,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604,0.046520,0.038520,0.085041
2010-01-01 05:00:00,-3.3,1,1,6,-2.762261,2.735587,9.187560,60.0,55.0,59.90,23.0,29.0,26.0,4.584604,0.046529,0.038529,0.085058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 19:00:00,-1.9,365,12,8756,-2.732825,2.778268,9.187559,60.0,55.0,59.85,23.0,29.0,26.0,4.588786,0.046443,0.038443,0.084887
2010-12-31 20:00:00,-2.2,365,12,8757,-2.736120,2.773985,9.187559,60.0,55.0,59.85,23.0,29.0,26.0,4.588786,0.046452,0.038452,0.084904
2010-12-31 21:00:00,-2.3,365,12,8758,-2.739409,2.769705,9.187559,60.0,55.0,59.85,23.0,29.0,26.0,4.588786,0.046461,0.038461,0.084921


In [276]:
# Tamb_hourly['E_loss_tot'].plot(figsize=(20,6))

In [277]:
# Tamb_hourly['E_loss_tot'].plot(figsize=(20,6))

## Save CSV version of the dataset

In [278]:
AVAILABLE_DATASET_PATHS = {"ospitaletto": ospitaletto2019.processed_dataset_path,
                           "miami_florida": noaa2010Dataset.processed_miami_fl_dataset_path, 
                           "fresno_california": noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           "olympia_washington": noaa2010Dataset.processed_olympia_wa_dataset_path,
                           "rochester_newyork": noaa2010Dataset.processed_rochester_ny_dataset_path}

Tamb_hourly.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])